In [6]:
import os
a=os.getenv('tezos_wallet_1')
print(a)

None


In [28]:
lambda_code='''(lambda %new_handler (unit) : unit
  {
    DROP;
    NIL operation;
    PAIR;
  })'''


In [185]:
from pytezos import pytezos
from dotenv import dotenv_values
# Load the .env file
env_vars = dotenv_values(".env")
private_key=env_vars.get("TEZOS_WALLET_1")
pytezos = pytezos.using(key=private_key, shell='https://ghostnet.ecadinfra.com')
# Define the contract address
contract_address = 'KT1VG3ynsnyxFGzw9mdBwYnyZAF8HZvqnNkw'
public_key="tz1T5kk65F9oZw2z1YV4osfcrX7eD5KtLj3c"
# Load the contract
contract = pytezos.contract(contract_address)
result = contract.registryCepDummy().send(min_confirmations=1)

In [144]:
import struct
import binascii

def pack_string(s):
    encoded = s.encode('utf-8')
    return struct.pack('>I', len(encoded)) + encoded

def pack_bytes(b):
    return struct.pack('>I', len(b)) + b

def generate_add_handler_metadata(name, handler_code, handler_check):
    return b'\x00' + pack_string(name) + pack_bytes(handler_code) + pack_bytes(handler_check)

def generate_remove_handler_metadata(name):
    return b'\x01' + pack_string(name)

def generate_execute_handler_metadata(handler_name, packed_argument):
    return b'\x02' + pack_string(handler_name) + pack_bytes(packed_argument)

# Example Michelson code (simplified for demonstration)
SIMPLE_HANDLER = """
{
  CDR ;
  NIL operation ;
  PAIR ;
}
"""

SIMPLE_HANDLER_CHECK = """
{
  DROP ;
  UNIT ;
}
"""

if __name__ == "__main__":
    # Generate add_handler metadata
    add_handler_metadata = generate_add_handler_metadata(
        "proposal_handler_1",
        SIMPLE_HANDLER.encode(),
        SIMPLE_HANDLER_CHECK.encode()
    )
    print("add_handler metadata (hex):")
    print(binascii.hexlify(add_handler_metadata).decode())

    # Generate Remove_handler metadata
    remove_handler_metadata = generate_remove_handler_metadata("proposal_handler_1")
    print("\nRemove_handler metadata (hex):")
    print(binascii.hexlify(remove_handler_metadata).decode())

    # Generate Execute_handler metadata
    execute_handler_metadata = generate_execute_handler_metadata(
        "proposal_handler_1",
        pack_bytes(b"Test string")  # Simulating a packed argument
    )
    print("\nExecute_handler metadata (hex):")
    print(binascii.hexlify(execute_handler_metadata).decode())

Add_handler metadata (hex):
000000001270726f706f73616c5f68616e646c65725f31000000280a7b0a2020434452203b0a20204e494c206f7065726174696f6e203b0a202050414952203b0a7d0a000000170a7b0a202044524f50203b0a2020554e4954203b0a7d0a

Remove_handler metadata (hex):
010000001270726f706f73616c5f68616e646c65725f31

Execute_handler metadata (hex):
020000001270726f706f73616c5f68616e646c65725f310000000f0000000b5465737420737472696e67


In [59]:
last_metadata=bytes("", "utf-8")
type(last_metadata)

bytes

In [ ]:
from dotenv import dotenv_values
# Load the .env file
env_vars = dotenv_values(".env")
env_vars.get("API_KEY_OPENAI")

In [150]:
import json
import struct

# Michelson schema string
michelson_schema_string = """
(or 
  (or 
    (pair %add_handler
      (pair (lambda %code
              (pair (pair (map %handler_storage string bytes) (bytes %packed_argument))
                    (pair %proposal_info
                        (address %from)
                        (nat %frozen_token)
                        (bytes %proposal_metadata)))
              (pair (pair (option %guardian address) (map %handler_storage string bytes))
                    (list %operations operation)))
            (lambda %handler_check (pair bytes (map string bytes)) unit))
      (string %name))
    (pair %execute_handler (string %handler_name) (bytes %packed_argument)))
  (string %remove_handler))
"""

# JSON data to encode
data_to_encode = {
  "add_handler": {
    "name": "example_handler",
    "code": {
      "prim": "PUSH",
      "args": [
        { "prim": "unit" },
        { "prim": "Unit" }
      ]
    },
    "handler_check": {
      "prim": "PUSH",
      "args": [
        { "prim": "unit" },
        { "prim": "Unit" }
      ]
    }
  }
}

# Function to encode Michelson data
def encode_michelson_data(data):
    def encode_prim(prim, args):
        encoded = b''
        if prim == 'Pair':
            # Assumes that args are encoded sequentially
            for arg in args:
                encoded += encode_michelson_data(arg)
        elif prim == 'string':
            encoded_string = args[0]
            length = len(encoded_string)
            encoded = struct.pack(">I", length) + encoded_string.encode()
        elif prim == 'bytes':
            encoded_bytes = args[0]
            encoded = bytes.fromhex(encoded_bytes)
        elif prim == 'unit':
            encoded = b'\x00'
        return encoded

    if isinstance(data, dict) and 'prim' in data:
        return encode_prim(data['prim'], data['args'])
    elif isinstance(data, dict) and 'string' in data:
        return encode_prim('string', [data['string']])
    elif isinstance(data, dict) and 'bytes' in data:
        return encode_prim('bytes', [data['bytes']])
    elif isinstance(data, dict) and 'unit' in data:
        return encode_prim('unit', [])

    raise ValueError("Unsupported Michelson data format")

# Wrap the data to match the expected format in the schema
wrapped_data = {
    "prim": "Pair",
    "args": [
        {"string": data_to_encode["add_handler"]["name"]},
        {
            "prim": "Pair",
            "args": [
                data_to_encode["add_handler"]["code"],
                data_to_encode["add_handler"]["handler_check"]
            ]
        }
    ]
}

# Encode data according to schema
packed_data = encode_michelson_data(wrapped_data)

# Convert to hex string
hex_string = packed_data.hex()

print("Packed Hex String:", hex_string)


Packed Hex String: 0000000f6578616d706c655f68616e646c6572


In [151]:
wrapped_data

{'prim': 'Pair',
 'args': [{'string': 'example_handler'},
  {'prim': 'Pair',
   'args': [{'prim': 'PUSH', 'args': [{'prim': 'unit'}, {'prim': 'Unit'}]},
    {'prim': 'PUSH', 'args': [{'prim': 'unit'}, {'prim': 'Unit'}]}]}]}

In [135]:
ctool.execute(proposal_metadata="7b2268616e646c65725f6e616d65223a202268656c6c6f5f6e65772068616e646c6572222c2022636f6465223a2022707573682022737472696e6722202248656c6c6f2c20776f726c64212220273b204e494c20276f7065726174696f6e20273b2050414952227d")

last metadata b''
curr metadata 7b2268616e646c65725f6e616d65223a202268656c6c6f5f6e65772068616e646c6572222c2022636f6465223a2022707573682022737472696e6722202248656c6c6f2c20776f726c64212220273b204e494c20276f7065726174696f6e20273b2050414952227d
*** USING CONTRACT INTERACTION TOOL ***
exception:  ({'id': 'proto.020-PsParisC.michelson_v1.script_rejected',
  'kind': 'temporary',
  'location': 294,
  'with': {'int': '111'}},)


In [93]:
import json

# Define the handler name, code, and handler check
handler_name = "hello_world_handler"
code = """
  { DROP ; PUSH string "Hello, world!" ; NIL operation ; PAIR }
"""
handler_check = """
  { DROP ; PUSH bool True }
"""

# Create the proposal_metadata dictionary
proposal_metadata = {
    "handler_name": handler_name,
    "code": code,
    "handler_check": handler_check
}

# Encode the metadata as JSON, then to UTF-8 bytes, and finally to a hex string
encoded_metadata = json.dumps(proposal_metadata).encode('utf-8').hex() # parameter that goes into the contract_interaction call
type(encoded_metadata)
again= bytes(encoded_metadata, "utf-8")


In [79]:
type(again)

bytes

In [88]:
from pytezos import pytezos
import json
import binascii
from dotenv import dotenv_values
# Load the .env file
rpc1="https://rpc.ghostnet.teztnets.com"
rpc2='https://ghostnet.ecadinfra.com'
env_vars = dotenv_values(".env")
private_key=env_vars.get("TEZOS_WALLET_1")
pytezos = pytezos.using(key=private_key, shell=rpc1)
# Define the contract address
contract_address = 'KT1VG3ynsnyxFGzw9mdBwYnyZAF8HZvqnNkw'
# Load the contract
contract = pytezos.contract(contract_address)
print("created contract")
from python.helpers.tool import Tool, Response
last_metadata=""
def call_contract(proposal_metadata="", **kwargs):
    global contract
    global last_metadata
    if proposal_metadata==last_metadata:
        return Response(message="Error: the metadata hasn't changed since last try.", break_loop=False)
    print("new unique bytes string")
    last_metadata=proposal_metadata
    print("*** USING CONTRACT INTERACTION TOOL ***")
    public_key="tz1T5kk65F9oZw2z1YV4osfcrX7eD5KtLj3c"
    mers=False
    something=None
    global stuff
    try:
        result = contract.propose(public_key, 10000,bytes(proposal_metadata, "utf-8")).send(min_confirmations=1)
        mers=True
    except Exception as e:
        print("exception: ",e)
        something=e
    if not mers:
        return something
    else:
        "Transaction submitted."

created contract


In [110]:
import requests

BASE_URL = "http://127.0.0.1:5000"

def call_hello_endpoint(metadata):
    response = requests.get(f"{BASE_URL}/hello", params={"metadata":metadata})
    print(response.json())
    return response


In [111]:
response=call_hello_endpoint("7b1268616e646c65725f6e616d65223a202268656c6c6f5f776f726c645f68616e646c6572222c2022636f6465223a20227b20204452504f503b205055534820737472696e67202248656c6c6f2c20776f726c64212220203b20524e494c20206f7065726174696f6e20203b2050414952227d222c202268616e646c65725f636865636b223a20227b20204452504f503b20505553482062756c6c20547275657d227d")
print(response)
print("type "+str(type(response)))

{'ERROR FROM CHAIN': "({'id': 'proto.020-PsParisC.michelson_v1.script_rejected',\n  'kind': 'temporary',\n  'location': 294,\n  'with': {'int': '111'}},)"}
<Response [200]>
type <class 'requests.models.Response'>


In [126]:
content = json.loads(response.content)

In [127]:
content['ERROR FROM CHAIN']

"({'id': 'proto.020-PsParisC.michelson_v1.script_rejected',\n  'kind': 'temporary',\n  'location': 294,\n  'with': {'int': '111'}},)"

In [95]:
again= bytes(encoded_metadata, "utf-8")
call_contract(proposal_metadata="a23fb2bfwebf2awv23va")

new unique bytes string
*** USING CONTRACT INTERACTION TOOL ***
exception:  ({'id': 'proto.020-PsParisC.michelson_v1.script_rejected',
  'kind': 'temporary',
  'location': 294,
  'with': {'int': '111'}},)


pytezos.rpc.errors.MichelsonError({'kind': 'temporary',
                                   'id': 'proto.020-PsParisC.michelson_v1.script_rejected',
                                   'location': 294,
                                   'with': {'int': '111'}})

In [49]:
from . import prepare_metadata

7b2268616e646c65725f6e616d65223a202268656c6c6f5f776f726c645f68616e646c6572222c2022636f6465223a20225c6e20207b2044524f50203b205055534820737472696e67205c2248656c6c6f2c20776f726c64215c22203b204e494c206f7065726174696f6e203b2050414952207d5c6e222c202268616e646c65725f636865636b223a20225c6e20207b2044524f50203b205055534820626f6f6c2054727565207d5c6e227d


In [44]:
import json
#https://www.youtube.com/watch?v=KHUn6KuNVMM
# Define the handler name, code, and handler check
handler_name = "hello_world_handler"
code = """
  { DROP ; PUSH string "Hello, world!" ; NIL operation ; PAIR }
"""
handler_check = """
  { DROP ; PUSH bool True }
"""

# Create the proposal_metadata dictionary
proposal_metadata = {
    "handler_name": handler_name,
    "code": code,
    "handler_check": handler_check
}

# Encode the metadata as JSON, then to UTF-8 bytes, and finally to a hex string
encoded_metadata = json.dumps(proposal_metadata).encode('utf-8').hex()

print(encoded_metadata)


7b2268616e646c65725f6e616d65223a202268656c6c6f5f776f726c645f68616e646c6572222c2022636f6465223a20225c6e20207b2044524f50203b205055534820737472696e67205c2248656c6c6f2c20776f726c64215c22203b204e494c206f7065726174696f6e203b2050414952207d5c6e222c202268616e646c65725f636865636b223a20225c6e20207b2044524f50203b205055534820626f6f6c2054727565207d5c6e227d


In [42]:
handler_name = "hello_world_handler"
code = """
  { DROP ; PUSH string "Hello, world!" ; NIL operation ; PAIR }
"""
handler_check = """
  { DROP ; PUSH bool True }
"""

# Create the proposal_metadata dictionary
geison = {
    "handler_name": handler_name,
    "code": code,
    "handler_check": handler_check
}

In [43]:
import json
data = {"key": "value"}
proposal_metadata = json.dumps(geison).encode('utf-8').hex()
proposal_metadata

'7b2268616e646c65725f6e616d65223a202268656c6c6f5f776f726c645f68616e646c6572222c2022636f6465223a20225c6e20207b2044524f50203b205055534820737472696e67205c2248656c6c6f2c20776f726c64215c22203b204e494c206f7065726174696f6e203b2050414952207d5c6e222c202268616e646c65725f636865636b223a20225c6e20207b2044524f50203b205055534820626f6f6c2054727565207d5c6e227d'

In [194]:
ceva={
  "prim": "Pair",
  "args": [
    {"string": "new_handler"},
    {
      "prim": "lambda",
      "args": [
        {"prim": "unit"},
        {"prim": "list", "args": [{"prim": "operation"}]},
        [
          {"prim": "DROP"},
          {"prim": "NIL", "args": [{"prim": "operation"}]},
          {"prim": "PAIR"}
        ]
      ]
    }
  ]
}


In [ ]:
from pytezos import michelson
code = '(Pair "new_handler" (lambda unit (list operation) { DROP; NIL operation; PAIR }))'
a=michelson.parse(code)

In [143]:
import struct
import binascii

def pack_string(s):
    encoded = s.encode('utf-8')
    return struct.pack('>I', len(encoded)) + encoded

def pack_bytes(b):
    return struct.pack('>I', len(b)) + b

def generate_add_handler_metadata(name, code, handler_check):
    return pack_string(name) + pack_bytes(code) + pack_bytes(handler_check)

def generate_remove_handler_metadata(name):
    return pack_string(name)

def generate_execute_handler_metadata(handler_name, packed_argument):
    return pack_string(handler_name) + pack_bytes(packed_argument)

# Generate add_handler metadata
add_handler_metadata = generate_add_handler_metadata(
    "my_handler",
    b'\x01\x02\x03',  # Placeholder for actual handler code
    b'\x04\x05\x06'   # Placeholder for actual handler_check code
)
print("add_handler metadata (hex):")
print(binascii.hexlify(add_handler_metadata).decode())

# Generate Remove_handler metadata
remove_handler_metadata = generate_remove_handler_metadata("handler_to_remove")
print("\nRemove_handler metadata (hex):")
print(binascii.hexlify(remove_handler_metadata).decode())

# Generate Execute_handler metadata
execute_handler_metadata = generate_execute_handler_metadata(
    "handler_to_execute",
    b'\x07\x08\x09'  # Placeholder for actual packed_argument
)
print("\nExecute_handler metadata (hex):")
print(binascii.hexlify(execute_handler_metadata).decode())

Add_handler metadata (hex):
0000000a6d795f68616e646c65720000000301020300000003040506

Remove_handler metadata (hex):
0000001168616e646c65725f746f5f72656d6f7665

Execute_handler metadata (hex):
0000001268616e646c65725f746f5f6578656375746500000003070809


In [7]:
import json
import binascii
lambda_code='''(lambda %new_handler (unit) : unit
  {
    DROP;
    NIL operation;
    PAIR;
  })'''
# Example Michelson code
michelson_code = lambda_code

lines = lambda_code.strip().split(';')
micheline = {'parameter': lines[0].split()[1], 'storage': lines[1].split()[1], 'code': lines[2].strip()}
micheline_json = json.dumps(micheline)
encoded_bytes = micheline_json.encode('utf-8')
hex_string = binascii.hexlify(encoded_bytes).decode('utf-8')
hex_string


'7b22706172616d65746572223a2022256e65775f68616e646c6572222c202273746f72616765223a20226f7065726174696f6e222c2022636f6465223a202250414952227d'

In [15]:
from pytezos import pytezos
from dotenv import dotenv_values
# Load the .env file
env_vars = dotenv_values(".env")
private_key=env_vars.get("TEZOS_WALLET_1")
pytezos = pytezos.using(key=private_key, shell='https://rpc.ghostnet.teztnets.com')
# Define the contract address
contract_address = 'KT1VG3ynsnyxFGzw9mdBwYnyZAF8HZvqnNkw' #DAO Thursday Bug Hunt
# contract_address = 'KT1Je4UBengJdFq28TnecpsmiR73Au4bG9U6' #simple function contract
# Load the contract
contract = pytezos.contract(contract_address)

In [ ]:
def call_contract(proposal_metadata):
    public_key="tz1T5kk65F9oZw2z1YV4osfcrX7eD5KtLj3c"
    mers=False
    something=None
    global stuff
    try:
        result = contract.propose(public_key, 10000, proposal_metadata ).send(min_confirmations=1)
        mers=True
    except Exception as e:
        print("exception: ",e)
        something=e
    if not mers:
        return something
    else:
        return "Transaction submitted."
call_contract(proposal_metadata=hex_string)

In [23]:
# import json
# import binascii
# lambda_code='''(lambda %new_handler (unit) : unit
#   {
#     DROP;
#     NIL operation;
#     PAIR;
#   })'''
# # Example Michelson code
# michelson_code = lambda_code

# lines = lambda_code.strip().split(';')
# micheline = {'parameter': lines[0].split()[1], 'storage': lines[1].split()[1], 'code': lines[2].strip()}
# micheline_json = json.dumps(micheline)
# encoded_bytes = micheline_json.encode('utf-8')
# hex_string = binascii.hexlify(encoded_bytes).decode('utf-8')
# print(hex_string)
testnet1="https://ghostnet.ecadinfra.com"
testnet2="https://rpc.ghostnet.teztnets.com"

from pytezos import pytezos
from dotenv import dotenv_values
# Load the .env file
env_vars = dotenv_values(".env")
private_key=env_vars.get("TEZOS_WALLET_1")
pytezos = pytezos.using(key=private_key, shell=testnet2)
# Define the contract address
contract_address = 'KT1VG3ynsnyxFGzw9mdBwYnyZAF8HZvqnNkw' #DAO Thursday Bug Hunt
# contract_address = 'KT1Je4UBengJdFq28TnecpsmiR73Au4bG9U6' #simple function contract
# Load the contract
contract = pytezos.contract(contract_address)

import json
import binascii
lambda_code='''(lambda %new_handler (unit) : unit
  {
    DROP;
    NIL operation;
    PAIR;
  })'''
# Example Michelson code
michelson_code = lambda_code

lines = lambda_code.strip().split(';')
micheline = {'parameter': lines[0].split()[1], 'storage': lines[1].split()[1], 'code': lines[2].strip()}
micheline_json = json.dumps(micheline)
encoded_bytes = micheline_json.encode('utf-8')
hex_string = binascii.hexlify(encoded_bytes).decode('utf-8')


def call_contract(proposal_metadata):
    global hex_string
    public_key="tz1T5kk65F9oZw2z1YV4osfcrX7eD5KtLj3c"
    mers=False
    something=None
    global stuff
    try:
        result = contract.propose(public_key, 10000, proposal_metadata ).send(min_confirmations=1)
        mers=True
    except Exception as e:
        print("exception: ",e)
        something=e
    if not mers:
        return something
    else:
        return "Transaction submitted."
call_contract(proposal_metadata=hex_string)

exception:  ({'id': 'proto.020-PsParisC.michelson_v1.script_rejected',
  'kind': 'temporary',
  'location': 294,
  'with': {'int': '111'}},)


pytezos.rpc.errors.MichelsonError({'kind': 'temporary',
                                   'id': 'proto.020-PsParisC.michelson_v1.script_rejected',
                                   'location': 294,
                                   'with': {'int': '111'}})

In [157]:
from pytezos import michelson

# Example Michelson code as a string
michelson_code = """
parameter string;
storage string;
code { DUP; DIP { CAR ; NIL string ; SWAP ; CONS } ; CDR ; CONS ; CONCAT ; NIL operation; PAIR }
"""

# Parse Michelson code
micheline = michelson.parse(michelson_code)

# Encode Micheline to bytes
encoded_bytes = michelson.pack(micheline)

print(encoded_bytes)


ImportError: cannot import name 'MichelsonRuntime' from 'pytezos.michelson' (/usr/local/lib/python3.10/dist-packages/pytezos/michelson/__init__.py)

In [156]:
from pytezos import michelson

# Define the Michelson code for the lambda function
michelson_code = '''
(Pair "my_custom_handler" (lambda unit (list operation) {
  DROP;
  NIL operation;
  PAIR;
}))
'''

# Encode the Michelson code to bytes
encoded_bytes = michelson.(michelson_code)
hex_encoded_bytes = encoded_bytes.hex()

print(hex_encoded_bytes)

In [155]:
import importlib
from python.tools.contract_interaction import ContractInteraction

In [99]:
result

<function OperationGroup.message at 0x7fbaac137010>
Get payload for the failing noop operation

:param block: Specify operation branch (default is genesis)
:returns: Message bytes

In [67]:
result


Properties
.key		tz1T5kk65F9oZw2z1YV4osfcrX7eD5KtLj3c
.shell		['https://ghostnet.ecadinfra.com']
.block_id	head

Payload
{'branch': 'BM8TXcSig7XYtRwbVKuhwSngFXtwgvXZSM4oQ9VWxtkxy1ekQXQ',
 'contents': [{'amount': '0',
               'counter': '10914431',
               'destination': 'KT1VG3ynsnyxFGzw9mdBwYnyZAF8HZvqnNkw',
               'fee': '353',
               'gas_limit': '683',
               'kind': 'transaction',
               'parameters': {'entrypoint': 'registryCepDummy',
                              'value': {'prim': 'Unit'}},
               'source': 'tz1T5kk65F9oZw2z1YV4osfcrX7eD5KtLj3c',
               'storage_limit': '100'}],
 'protocol': 'PsParisCZo7KAh1Z1smVd9ZMZ1HHn5gkzbM94V3PLCpknFWhUAi',
 'signature': 'sigsMJAfh2GKcQu9rSee2AivVktaQfeAQzAobE2rFTsKKn2CXmzkiLRcJ9WX4Z2627tY8frzb2VLzawvDnMXEtzW6rXa4yab'}

Hash
oogFfyJVk2yV9DgzhyF2PoKqv5iY9dbojyFcr6Wm5bAKP3ZA7F6
.activate_account()
.autofill()
.ballot()
.binary_payload()
.delegation()
.double_baking_evidence()
.dou